In [8]:
import json
from sklearn.model_selection import train_test_split

In [9]:
import json
import random
from itertools import permutations

def convert_label_studio_to_re_format(
    input_file, 
    output_file, 
    negative_ratio=0.5 
):
    """
    Chuyển đổi JSON export từ Label Studio sang định dạng train cho RBERT.
    Đã sửa để hỗ trợ cả Single Object JSON và List JSON.
    """
    
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            ls_data = json.load(f)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {input_file}")
        return []

    # --- SỬA LỖI QUAN TRỌNG: Chuẩn hóa dữ liệu về dạng List ---
    # Nếu file json chỉ chứa 1 object (bắt đầu bằng { "id":... }), ta gói nó vào list
    if isinstance(ls_data, dict):
        ls_data = [ls_data]
    
    final_data = []
    stats = {"positive": 0, "negative": 0, "relations": {}}

    for task in ls_data:
        # 1. Lấy văn bản gốc
        # Kiểm tra kỹ key chứa text (thường là 'text' hoặc 'sentence' tùy config Label Studio)
        data_block = task.get('data', {})
        text = data_block.get('text') or data_block.get('sentence')
        
        if not text:
            print(f"Bỏ qua Task ID {task.get('id', 'Unknown')}: Không tìm thấy text.")
            continue

        # 2. Parse Entities (Nodes)
        entities = {}
        annotations = task.get('annotations', [])
        if not annotations:
            continue
            
        # Lấy annotation đầu tiên (thường là bản mới nhất hoặc ground_truth)
        result = annotations[0].get('result', [])
        
        # Bước 2a: Map ID -> Entity Info
        for item in result:
            if item['type'] == 'labels':
                entity_id = item['id']
                value = item['value']
                
                # Lấy nhãn đầu tiên trong mảng labels
                label_type = value['labels'][0] if value.get('labels') else "Unknown"

                entities[entity_id] = {
                    'text': value['text'],
                    'start': value['start'],
                    'end': value['end'],
                    'type': label_type  # Quan trọng cho Type-Aware Model
                }

        # Bước 2b: Lấy Relations (Edges)
        existing_relations = set() 
        
        for item in result:
            if item['type'] == 'relation':
                from_id = item['from_id']
                to_id = item['to_id']
                relation_label = item['labels'][0] if item.get('labels') else "NO_RELATION"
                
                if from_id in entities and to_id in entities:
                    subj = entities[from_id]
                    obj = entities[to_id]
                    
                    sample = {
                        "text": text,
                        "relation": relation_label,
                        "subj": subj,
                        "obj": obj
                    }
                    final_data.append(sample)
                    existing_relations.add((from_id, to_id))
                    
                    stats["positive"] += 1
                    stats["relations"][relation_label] = stats["relations"].get(relation_label, 0) + 1

        # Bước 3: Negative Sampling (Tạo mẫu No_Relation)
        entity_ids = list(entities.keys())
        all_pairs = list(permutations(entity_ids, 2)) 
        
        potential_negatives = []
        for e1_id, e2_id in all_pairs:
            if (e1_id, e2_id) not in existing_relations:
                potential_negatives.append((e1_id, e2_id))
        
        # Logic lấy số lượng negative sample
        num_neg_to_take = int(len(existing_relations) * negative_ratio)
        
        # Nếu có entity nhưng chưa có relation nào, vẫn lấy 1 mẫu negative để model học
        if len(existing_relations) == 0 and len(potential_negatives) > 0 and negative_ratio > 0:
             num_neg_to_take = 1
        # Nếu tính ra 0 nhưng user muốn lấy (negative_ratio > 0) và có relations, tối thiểu lấy 1
        elif len(existing_relations) > 0 and num_neg_to_take == 0 and negative_ratio > 0:
             num_neg_to_take = 1

        random.shuffle(potential_negatives)
        selected_negatives = potential_negatives[:num_neg_to_take]
        
        for e1_id, e2_id in selected_negatives:
            subj = entities[e1_id]
            obj = entities[e2_id]
            
            sample = {
                "text": text,
                "relation": "No_Relation",
                "subj": subj,
                "obj": obj
            }
            final_data.append(sample)
            stats["negative"] += 1

    # Lưu file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)

    print("=== Conversion Completed ===")
    print(f"Total samples generated: {len(final_data)}")
    print(f"Positive samples: {stats['positive']}")
    print(f"Negative samples: {stats['negative']}")
    print("Relation Types Breakdown:", stats["relations"])
    return final_data

# --- CHẠY VỚI DỮ LIỆU CỦA BẠN ---
# Lưu đoạn JSON bạn đưa vào file tên 'dataset2.json' trước khi chạy
convert_label_studio_to_re_format("dataset2.json", "train_data_fixed.json", negative_ratio=1.0)

=== Conversion Completed ===
Total samples generated: 1185
Positive samples: 488
Negative samples: 697
Relation Types Breakdown: {'TREATED_WITH': 98, 'CAUSES': 145, 'REVEALS': 23, 'HAS_MANIFESTATION': 210, 'NO_RELATION': 12}


[{'text': 'bảng phân loại teo thực quản của gross : giúp quyết định điều trị và tiên lượng bệnh nhóm a : teo thực quản không có lỗ dò ( 8% ) nhóm b : teo thực quản có lỗ dò ở đầu gần thực quản – khí quản ( &lt; 1% ) nhóm c : teo thực quản có lỗ dò ở đầu xa thực quản - khí quản ( 87% ) nhóm d : teo thực quản có lỗ dò ở hai đầu thực quản – khí quản ( &lt; 1% ) nhóm e : dò thực quản - khí quản không teo ( dò dạng h ) ( 4% ) nhóm f : hẹp thực quản ( &lt;1% ) các biện pháp điều trị bệnh teo thực quản nguyên tắc điều trị teo thực quản : chuyển viện sớm đến bệnh viện có khoa phẫu thuật nhi .',
  'relation': 'TREATED_WITH',
  'subj': {'text': 'teo thực quản',
   'start': 510,
   'end': 523,
   'type': 'DISEASE'},
  'obj': {'text': 'phẫu thuật',
   'start': 564,
   'end': 574,
   'type': 'TREATMENT'}},
 {'text': 'bảng phân loại teo thực quản của gross : giúp quyết định điều trị và tiên lượng bệnh nhóm a : teo thực quản không có lỗ dò ( 8% ) nhóm b : teo thực quản có lỗ dò ở đầu gần thực quản – 

In [10]:
with open('train_data_fixed.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

labels = [item['relation'] for item in data]

# 2. Split train/temp (80/20)
train_data, temp_data = train_test_split(
    data, 
    test_size=0.2, 
    random_state=42, 
    stratify=labels
)

# 3. Split dev/test (50/50)
temp_labels = [item['relation'] for item in temp_data]
dev_data, test_data = train_test_split(
    temp_data, 
    test_size=0.5, 
    random_state=42, 
    stratify=temp_labels
)

# 4. Save
def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

save_json(train_data, 'train.json')
save_json(dev_data, 'dev.json')
save_json(test_data, 'test.json')

print(f"Train: {len(train_data)}, Dev: {len(dev_data)}, Test: {len(test_data)}")

Train: 948, Dev: 118, Test: 119
